# Feast Basic Customer Transactions Example

This is a minimal example of using Feast. In this example we will
1. Create a synthetic customer feature dataset
2. Register a feature set to represent these features in Feast
3. Ingest these features into Feast
4. Create a feature query to retrieve historical feature data
5. Create a feature query to retrieve online feature data

### 1. Clone Feast and install all dependencies

In [ ]:
!rm -rf feast \
&& git clone https://github.com/gojek/feast.git \
&& cd feast/sdk/python/ &&  pip install --upgrade --quiet -e . \
&& pip install --quiet --upgrade pandas numpy protobuf

### 2. Import necessary modules

In [8]:
import pandas as pd
import numpy as np
from pytz import timezone, utc
from feast import Client, FeatureSet, Entity, ValueType
from feast.serving.ServingService_pb2 import GetOnlineFeaturesRequest
from feast.types.Value_pb2 import Value as Value
from google.protobuf.duration_pb2 import Duration
from datetime import datetime, timedelta
from random import randrange
import random

### 3. Configure Feast services and connect the Feast client

In [9]:
CORE_URL = 'localhost:6565'
ONLINE_SERVING_URL = 'localhost:6566'
BATCH_SERVING_URL = 'localhost:6567'

In [10]:
client = Client(core_url=CORE_URL, serving_url=BATCH_SERVING_URL) # Connect to Feast Core

### 4. Create synthetic customer features

Here we will create customer features for 5 customers over a month. Each customer will have a set of features for every day.

In [24]:
days = [datetime.utcnow().replace(hour=0, minute=0, second=0, microsecond=0).replace(tzinfo=utc) \
        - timedelta(day) for day in range(31)]

customers = [1001, 1002, 1003, 1004, 1005]

In [25]:
customer_features = pd.DataFrame(
    {
        "datetime": [day for day in days for customer in customers],
        "customer_id": [customer for day in days for customer in customers],
        "daily_transactions": [np.random.rand() * 10 for _ in range(len(days) * len(customers))],
        "total_transactions": [np.random.randint(100) for _ in range(len(days) * len(customers))],
    }
)

print(customer_features.head(10))

                   datetime  customer_id  daily_transactions  \
0 2019-12-08 00:00:00+00:00         1001            1.418540   
1 2019-12-08 00:00:00+00:00         1002            4.231620   
2 2019-12-08 00:00:00+00:00         1003            2.125182   
3 2019-12-08 00:00:00+00:00         1004            6.328034   
4 2019-12-08 00:00:00+00:00         1005            0.009291   
5 2019-12-07 00:00:00+00:00         1001            1.199422   
6 2019-12-07 00:00:00+00:00         1002            4.296611   
7 2019-12-07 00:00:00+00:00         1003            8.178577   
8 2019-12-07 00:00:00+00:00         1004            4.351829   
9 2019-12-07 00:00:00+00:00         1005            5.066446   

   total_transactions  
0                  34  
1                  93  
2                  56  
3                  95  
4                  49  
5                  16  
6                  90  
7                  14  
8                  22  
9                  44  


### 5. Create feature set for customer features

Now we will create a feature set for these features. Feature sets are essentially a schema that represent
feature values. Feature sets allow Feast to both identify feature values and their structure. 

In this case we need to define any entity columns as well as the maximum age. The entity column in this case is "customer_id". Max age is set to 1 day (defined in seconds). This means that for each feature query during retrieval, the serving API will only retrieve features up to a maximum of 1 day per provided timestamp and entity combination. 

In [13]:
customer_fs = FeatureSet(
    "customer_transactions",
    max_age=Duration(seconds=86400),
    entities=[Entity(name='customer_id', dtype=ValueType.INT64)]
)

Here we are automatically inferring the schema from the provided dataset

In [26]:
customer_fs.infer_fields_from_df(customer_features, replace_existing_features=True)

Feature daily_transactions (ValueType.DOUBLE) added from dataframe.
 Feature total_transactions (ValueType.INT64) added from dataframe.
 


### 6. Register feature set with Feast Core

The apply() method will register the provided feature set with Feast core, allowing users to retrieve features from this feature set

In [16]:
client.apply(customer_fs)

No change detected or applied: customer_transactions


We test the retrieval of this feature set object (not its data), to ensure that we have the latest version

In [17]:
customer_fs = client.get_feature_set("customer_transactions")
print(customer_fs)

{
  "name": "customer_transactions",
  "version": 3,
  "entities": [
    {
      "name": "customer_id",
      "valueType": "INT64"
    }
  ],
  "features": [
    {
      "name": "daily_transactions",
      "valueType": "DOUBLE"
    },
    {
      "name": "total_transactions",
      "valueType": "INT64"
    }
  ],
  "maxAge": "86400s",
  "source": {
    "type": "KAFKA",
    "kafkaSourceConfig": {
      "bootstrapServers": "10.202.112.114:9092",
      "topic": "feast-features"
    }
  }
}


### 7. Ingest data into Feast for a feature set

In [27]:
client.ingest("customer_transactions", customer_features)

  0%|          | 0/155 [00:00<?, ?rows/s]


(ingest table to kafka) Ingestion started for customer_transactions:3


  0%|          | 0/155 [00:00<?, ?rows/s]


Ingestion statistics:
Success: 155/155


### 8. Create a batch retrieval query

In order to retrieve historical feature data, the user must provide an entity_rows dataframe. This dataframe contains a combination of timestamps and entities. In this case, the user must provide both customer_ids and timestamps. 

We will randomly generate timestamps over the last 30 days, and assign customer_ids to them. When these entity rows are sent to the Feast Serving API to retrieve feature values, along with a list of feature ids, Feast is then able to attach the correct feature values to each entity row. The one exception is if the feature values fall outside of the maximum age window.

In [30]:
event_timestamps = [datetime.utcnow().replace(tzinfo=utc) - timedelta(days=randrange(15), hours=randrange(24), minutes=randrange(60)) for day in range(30)]

entity_rows = pd.DataFrame(
    {
        "datetime": event_timestamps,
        "customer_id": [customers[idx % len(customers)] for idx in range(len(event_timestamps))],
    }
)

print(entity_rows.head(10))

                          datetime  customer_id
0 2019-11-27 02:02:25.325169+00:00         1001
1 2019-12-06 00:27:25.325198+00:00         1002
2 2019-12-06 15:04:25.325205+00:00         1003
3 2019-12-02 04:57:25.325210+00:00         1004
4 2019-12-04 11:16:25.325215+00:00         1005
5 2019-12-08 03:53:25.325220+00:00         1001
6 2019-12-07 21:49:25.325226+00:00         1002
7 2019-12-07 16:14:25.325231+00:00         1003
8 2019-11-29 06:05:25.325236+00:00         1004
9 2019-11-27 15:01:25.325241+00:00         1005


### 9. Retrieve historical/batch features

In [32]:
job = client.get_batch_features(
                            feature_ids=[
                                f"customer_transactions:{customer_fs.version}:daily_transactions", 
                                f"customer_transactions:{customer_fs.version}:total_transactions", 
                               ],
                            entity_rows=entity_rows
                         )
df = job.to_dataframe()
print(df.head(10))

                                   uuid                  event_timestamp  \
0  9dd6adfe-c8fb-472f-9db6-913827f240dd 2019-11-27 02:02:25.325169+00:00   
1  5a84f26d-cb3c-4767-8d71-b3cb76e060b4 2019-12-08 03:53:25.325220+00:00   
2  d458db71-ff1b-4520-8d2d-cba03bed3118 2019-12-01 18:50:25.325246+00:00   
3  cf6fb51b-4396-442a-a5b7-615550755ac7 2019-12-05 14:35:25.325271+00:00   
4  b92b0eed-f2e2-49e3-8f1b-b1d3c3cf3072 2019-12-03 10:17:25.325295+00:00   
5  af4f716a-6c89-48ec-81c8-cb5bbd13e610 2019-11-27 21:01:25.325320+00:00   
6  c50e69f0-32ed-4af2-93f5-89bd202500ec 2019-12-06 00:27:25.325198+00:00   
7  fde6a2bb-91ce-4c9f-9112-9e579dcbde1e 2019-12-07 21:49:25.325226+00:00   
8  c281728f-64a4-4808-91f4-c75d2a2b0329 2019-11-29 16:51:25.325251+00:00   
9  81e9007c-64b2-48db-9c34-1923a3fd088e 2019-12-06 15:36:25.325275+00:00   

   customer_id  customer_transactions_v3_daily_transactions  \
0         1001                                          NaN   
1         1001                       

### 10. Retrieve online features

The process of retrieving features from the online API is very similar to that of the batch API. The only major difference is that users do not have to provide timestamps (only the latest features are returned, as long as they are within the maximum age window)

In [36]:
online_client = Client(core_url=CORE_URL, serving_url=ONLINE_SERVING_URL)

The example below retrieves online features for a single customer: "1001"

In [37]:
online_features = online_client.get_online_features(
    feature_ids=[
        f"customer_transactions:{customer_fs.version}:daily_transactions",
        f"customer_transactions:{customer_fs.version}:total_transactions",
    ],
    entity_rows=[
        GetOnlineFeaturesRequest.EntityRow(
            fields={
                "customer_id": Value(
                    int64_val=1001)
            }
        )
    ],
)

ConnectionError: Connection timed out while attempting to connect to Feast Serving gRPC server 10.202.112.114:6566 

In [ ]:
print(online_features)